In [1]:
# mkdir if not exist 'proposals_out_2023_07_21'
import os
if not os.path.exists('votes_out_2023_07_21'):
    os.makedirs('votes_out_2023_07_21')

In [2]:
# read in ../../DATA/deployments/snapshot_deployments.csv
import pandas as pd
df = pd.read_parquet('../../proposals/snapshot/snapshot_proposals.parquet')
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count,title,description,start,end
0,snapshot,modeplc.eth,0xd8c75dbcb32a6c82e049b482ecaa9706a78c2fc91d1a...,0x2b49F45B80d7AD5C3Af9C96B5B664A953979d16A,2021-11-29 14:30:53,3,Is SocialFi or GameFi next hustle?,,2021-11-29 04:00:00,2022-06-30 04:00:00
1,snapshot,modeplc.eth,0xf4fbef4f0a128f233b6da3b744d10d3b7a3c74d4b048...,0x2b49F45B80d7AD5C3Af9C96B5B664A953979d16A,2021-11-29 14:23:09,3,Is SocialFi or GameFi next hustle?,,2021-11-29 04:00:00,2022-06-30 04:00:00
2,snapshot,brummer.eth,0xdd9a17031b8e109c04cef929129692e1632e8cfc40a6...,0x9349738c1491d4DDDfddDc3A4Cb0E3e709934C63,2021-11-29 14:05:19,5,Will Elon be the leadship in Crypto?,,2021-11-29 04:00:00,2022-06-30 04:00:00
3,snapshot,brightunion.eth,0xc8c4f6054e9f9e1255768752ee07aaa9bdca15c95749...,0xB2b8DE16675ef164e51910BDf15e62F2f80f7f4c,2021-11-29 13:47:35,14,[BRIGHT-1] Chain for Bright Union NFT collection,As part of the BRIGHT loyalty program NFTs fro...,2021-11-29 08:00:00,2021-12-03 11:00:00
4,snapshot,dystomice.eth,0xb7ed8c45ce8f8c23e52acc057ddc3d987d012532df7f...,0xAB45d5A30034e7959968cB7362f494bb95c19aB7,2021-11-29 09:57:18,24,DAO Leadership Nominations,"This vote is open to DYSTOMICE, SPACEMICE and ...",2021-11-29 20:00:00,2021-12-01 20:00:00


In [3]:
df.votes_count.describe()

count    1.165540e+05
mean     4.378009e+02
std      1.141047e+04
min      0.000000e+00
25%      1.000000e+00
50%      4.000000e+00
75%      2.200000e+01
max      2.765849e+06
Name: votes_count, dtype: float64

In [4]:
# sum of votes_count
df.votes_count.sum() / 5000
# we'll need this number of requests to get all votes

10205.4892

In [5]:
df.sort_values(by='votes_count', ascending=False).head(10)

,platform,platform_deployment_id,proposal_id,author,date,votes_count,title,description,start,end
108160,snapshot,linea-build.eth,0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96...,0x4214D1Abb6954daB1c61918a0CB10E006FCD33e3,2023-05-23 14:19:43,2765849,What is the Linea SWAG that you would like to ...,Date: 05-24-2023\nAuthor: Linea Team\nProposal...,2023-05-23 14:19:33,2023-05-29 04:00:00
68093,snapshot,stgdao.eth,0xc18c4911e19adf4bd54c6ad142bb08bc81fd44dee723...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-10 15:42:20,514470,Deprecate the WOO liquidity pool on Fantom,"-Background-\nOn the 6th of July 2023, cross-c...",2023-08-10 15:42:20,2023-08-13 15:42:20
68094,snapshot,stgdao.eth,0xcfc335d45421a2d3b5f64d5ee19770ab103f539cedc2...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-10 15:30:36,510861,Make Aura and Balancer the STG Liquidity Hub o...,"-Background-\nCurrently, Stargate has protocol...",2023-08-10 15:30:35,2023-08-13 15:30:35
68095,snapshot,stgdao.eth,0xd6bb6427e285ae4e269ebb9f4d0396808d2de356c654...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-10 15:26:35,510523,Beefy provides $OP incentives for STG liquidity.,Background\nBeefy is a multi-chain yield optim...,2023-08-10 15:26:34,2023-08-13 15:26:34
67741,snapshot,stgdao.eth,0x2eff53c9e9f0bf19a98345d1d32103ec0ec859d81978...,0xd54A2392c8F2AD3a4e927Cf24EC12797f2503C7b,2023-09-04 22:05:05,472911,Stargate Foundation 12-month Budget Proposal,**Issue Statement**\n\nStargate Finance has gr...,2023-09-04 22:05:05,2023-09-07 22:05:05
61713,snapshot,stgdao.eth,0xed5fe9d40c9e82a53c548d4458fdca91d6d26c12cc34...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-07-05 19:30:18,470709,Stargate Deployment On Kava,Background \nThe Kava Network is a Layer-1 blo...,2023-07-05 19:30:18,2023-07-08 19:30:18
67742,snapshot,stgdao.eth,0x3ab405d7d396f0b3bf04b966dd12b06ad5e81d3cf583...,0xd54A2392c8F2AD3a4e927Cf24EC12797f2503C7b,2023-09-04 21:50:33,470250,Lower TheAptosBridge Block Confirmations and A...,**Background**\n\nPer a community vote (https:...,2023-09-04 21:50:33,2023-09-07 21:50:33
67833,snapshot,stgdao.eth,0xe8e92474bbd4ad0aa011cf69ad1873a9e30ab9c431f7...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-23 18:38:45,456653,Renewed Liquidity Incentivization Mandate,"--Background--\nOn May 6, 2022, Stargate DAO v...",2023-08-23 18:38:45,2023-08-26 18:38:45
67835,snapshot,stgdao.eth,0x6078a960e2cb6b518dddc3773f7f40b30ab76bc6f2f3...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-23 18:27:20,456232,Stargate Deployment on IDEX/XCHAIN,TL;DR\n-Deploy Stargate on IDEX & XCHAIN as th...,2023-08-23 18:27:19,2023-08-26 18:27:19
67834,snapshot,stgdao.eth,0x73917ac1c09063a4776aea60176817f3aa3ee96b3a54...,0xCff5be1cf3C7Cfe073Ade82d1dC27be33c998D6e,2023-08-23 18:30:22,455835,Transfer Ownership of TheAptosBridge to Stargate,"In late 2022, LayerZero launched TheAptosBridg...",2023-08-23 18:30:21,2023-08-26 18:30:21


In [6]:
# remove the space_ids that have no proposals
df = df[df.votes_count > 0]
proposals_and_votes_count = df[['proposal_id', 'votes_count']].sort_values(by='votes_count', ascending=False)
# make these into a list of tuples
proposals_and_votes_count = list(proposals_and_votes_count.itertuples(index=False, name=None))
proposals_and_votes_count[:5]

[('0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96c3ab22abaa1c7244648e',
  2765849),
 ('0xc18c4911e19adf4bd54c6ad142bb08bc81fd44dee723e520c511c2a9414d5920',
  514470),
 ('0xcfc335d45421a2d3b5f64d5ee19770ab103f539cedc240ba937691093b392bc4',
  510861),
 ('0xd6bb6427e285ae4e269ebb9f4d0396808d2de356c654c19877402da4e3e8c44a',
  510523),
 ('0x2eff53c9e9f0bf19a98345d1d32103ec0ec859d81978eecacb38db16be33b94b',
  472911)]

In [7]:
# make large_proposals_and_votes where votes_count > 5000
large_proposals_and_votes = [x for x in proposals_and_votes_count if x[1] > 5000]
# make normal_proposals_and_votes where votes_count <= 5000
normal_proposals_and_votes = [x for x in proposals_and_votes_count if x[1] <= 5000]

In [8]:
# save the large_proposals_and_votes
import pickle
with open('large_proposals_and_votes.pkl', 'wb') as f:
    pickle.dump(large_proposals_and_votes, f)

In [9]:
# make groups of rows where the proposals_count in the group sums to less than 5000
groups_of_proposal_ids_that_sum_to_less_than_5000 = []
while len(normal_proposals_and_votes) > 0:
    group = []
    total = 0

    while total < 5000:
        if normal_proposals_and_votes[0][1] + total > 5000:
            break
        row = normal_proposals_and_votes.pop(0)
        group.append(row)
        total += row[1]
        if len(normal_proposals_and_votes) == 0:
            break
    groups_of_proposal_ids_that_sum_to_less_than_5000.append(group)

groups_of_proposal_ids_that_sum_to_less_than_5000

[[('0x6bccf3a1f627aa4056c921f8551683c82f3a4e0d3c831840d5738c1748f78790',
   4991)],
 [('0x643a0ee899adf7d52a5e8d4fff6040eca35d81b6c1f4cbbb557d8e721637b5aa',
   4987)],
 [('0x549fa2186f321a4e8a07e0d5e82f85f3e2c83189cfaee678a6df144bb67d8f54',
   4952)],
 [('0xbab5cd9d9cc5123ea5da8eb45e321463f1a0fd2543dc17587db1f24099d90dfc',
   4934)],
 [('0x707643dd2c2b941c7274f26a0362922198912e8549050ae9f2d1d9d71ada04bb',
   4918)],
 [('0xe9ff93315d1ce1ee0fe746a3074b866566de096cf912d45350f35e4fe019f502',
   4880)],
 [('0x4b362044000714197ab43d5f2155977976dcc7cf1cb28d945e1168ae7ac61f90',
   4872)],
 [('0x381c007c5d0b05adc5eb7fc5daddf3164c6d24a0c51b163046f1192f7a67fced',
   4814)],
 [('0xf56c77c8a01b329307868bda6b769e9f6f55397f9eb90d289a755c8531936e5e',
   4791)],
 [('0xc5de3d224378996b083d61501f9323c4abd5d78cd05c3268d21cb4057cdb8341',
   4769)],
 [('0x288caef0d79e5883884324b90daa3c5550135ea0c78738e7ca2363243340c2da',
   4716)],
 [('0xe81f852d90ba80929b1f19683da14b334d63b31cb94e53249b8caed715475693',
   

In [10]:
sums = [sum([x[1] for x in group]) for group in groups_of_proposal_ids_that_sum_to_less_than_5000]
sums

[4991,
 4987,
 4952,
 4934,
 4918,
 4880,
 4872,
 4814,
 4791,
 4769,
 4716,
 4708,
 4703,
 4702,
 4689,
 4687,
 4674,
 4665,
 4660,
 4652,
 4621,
 4618,
 4607,
 4565,
 4537,
 4523,
 4518,
 4512,
 4502,
 4500,
 4477,
 4472,
 4468,
 4467,
 4459,
 4448,
 4435,
 4419,
 4419,
 4418,
 4416,
 4406,
 4400,
 4368,
 4342,
 4329,
 4313,
 4312,
 4242,
 4215,
 4155,
 4144,
 4137,
 4129,
 4126,
 4121,
 4113,
 4109,
 4103,
 4096,
 4093,
 3979,
 3979,
 3976,
 3971,
 3970,
 3969,
 3958,
 3951,
 3944,
 3931,
 3918,
 3891,
 3886,
 3875,
 3875,
 3848,
 3820,
 3807,
 3783,
 3769,
 3756,
 3746,
 3735,
 3726,
 3711,
 3706,
 3706,
 3704,
 3701,
 3701,
 3699,
 3697,
 3671,
 3665,
 3655,
 3650,
 3650,
 3644,
 3613,
 3602,
 3600,
 3588,
 3584,
 3576,
 3568,
 3562,
 3547,
 3545,
 3527,
 3514,
 3512,
 3501,
 3478,
 3461,
 3451,
 3447,
 3441,
 3429,
 3408,
 3367,
 3365,
 3359,
 3349,
 3332,
 3326,
 3323,
 3319,
 3315,
 3312,
 3311,
 3308,
 3303,
 3301,
 3292,
 3289,
 3278,
 3278,
 3276,
 3273,
 3270,
 3265,
 3261,

In [11]:
len(groups_of_proposal_ids_that_sum_to_less_than_5000)

1346

In [12]:
# make into a list of strings for the graphql query
groups_of_proposal_ids_that_sum_to_less_than_5000_strings = []
for group in groups_of_proposal_ids_that_sum_to_less_than_5000:
    group_of_space_ids = [str(x[0]) for x in group]
    query_string = ", ".join([f'"{sid}"' for sid in group_of_space_ids])
    groups_of_proposal_ids_that_sum_to_less_than_5000_strings.append(query_string)

groups_of_proposal_ids_that_sum_to_less_than_5000_strings[0]

'"0x6bccf3a1f627aa4056c921f8551683c82f3a4e0d3c831840d5738c1748f78790"'

In [13]:
def make_query(proposal_ids: str, skip: int) -> str:
    first = 1000
    """
    * platform
    * platform_deployment_id
    * proposal_id
    * vote_id
    * voter -- the id of the person who made the vote
    * date -- this should be as a pandas date, not as an epoch
    * choice -- this should be a string as this is not always true/false
    * weight -- the vote weight
    """
    base = """
       query {
         votes(where: {proposal_in: [%s]}, first: %s, skip: %s, orderDirection: desc, orderBy: "created") {
            space {
              id
            }
            proposal {
              id
            }
            id
            voter
            created
            vp_by_strategy
            vp
          }
        }
    """
    return base % (proposal_ids, first, skip)


In [14]:
import requests
import backoff
URL = "https://hub.snapshot.org/graphql"

# make custom SnapshotException
class SnapshotException(Exception):

    def __init__(self, message='SnapshotException'):
        self.message = message
        super().__init__(self.message)

    def __str__(self):
        return f"SnapshotException: {self.message}"


@backoff.on_exception(backoff.expo,
                      requests.exceptions.RequestException)
@backoff.on_exception(backoff.expo,
                      SnapshotException)
def query_snapshot(query):
    r = requests.post(URL, json={'query': query})
    data = r.json()
    if data.get('error'):
        raise SnapshotException
    return data['data']['votes']

In [15]:
# make csv log.txt in the votes_out_2023_07_21 directory if it doesn't exist
if not os.path.exists('votes_out_2023_07_21/log.csv'):
    with open('votes_out_2023_07_21/log.txt', 'w') as f:
        f.write('')

In [16]:
import json

def process_proposal_group(count, proposal_id_group):
    # read log as list of strings
    with open('votes_out_2023_07_21/log.txt', 'r') as f:
        log = f.readlines()

    # make pagination logic
    skip = 0
    has_more = True
    votes_processed = 0
    while has_more:
        filename = f'votes_out_2023_07_21/votes_{count}_{skip}.json'
        if os.path.exists(filename) or filename + '\n' in log:
            # print('skipping', filename)
            skip += 1000
            if skip >= 5000:
                has_more = False
            continue
        query = make_query(proposal_id_group, skip)
        votes = query_snapshot(query)
        votes_processed += len(votes)
        print('got', len(votes), 'votes for', count, skip)
        # write filename to log
        with open('votes_out_2023_07_21/log.txt', 'a') as f:
            f.write(filename + '\n')
        # save votes to file in the output dir
        with open(filename, 'w') as f:
            json.dump(votes, f)
        if len(votes) == 1000:
            skip += 1000
        else:
            has_more = False
    
    return votes_processed

In [17]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(process_proposal_group, count, proposal_id_group) for count, proposal_id_group in enumerate(groups_of_proposal_ids_that_sum_to_less_than_5000_strings)]

    # Wait for all threads to complete
    votes_processed_list = [future.result() for future in futures]

TODO: need to handle the large ones, too, with groups of dates